In [1]:
import numpy as np
import pandas as pd

In [2]:
df  =  pd.read_csv('titanic_train.csv')

In [3]:
df.shape

(891, 12)

In [4]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [5]:
df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [6]:
df.loc[:, ['Survived','Pclass','Sex', 'Embarked']]

,Survived,Pclass,Sex,Embarked
0,0,3,male,S
1,1,1,female,C
2,1,3,female,S
3,1,1,female,S
4,0,3,male,S
...,...,...,...,...
886,0,2,male,S
887,1,1,female,S
888,0,3,female,S
889,1,1,male,C


In [7]:
df  = df.loc[df.Embarked.notna(), ['Survived','Pclass','Sex', 'Embarked']]

In [8]:
df.shape

(889, 4)

In [9]:
df.head()

,Survived,Pclass,Sex,Embarked
0,0,3,male,S
1,1,1,female,C
2,1,3,female,S
3,1,1,female,S
4,0,3,male,S


In [10]:
X  =df.loc[:, ['Pclass']]

In [11]:
y = df.Survived

In [12]:
X.shape

(889, 1)

In [13]:
y.shape

(889,)

In [14]:
from sklearn.linear_model import LogisticRegression


In [15]:
logmodel  =  LogisticRegression(solver = 'lbfgs')

In [16]:
from sklearn.model_selection import cross_val_score

In [17]:
cross_val_score(logmodel, X,y, cv=5, scoring='accuracy')

array([0.6011236 , 0.6741573 , 0.6741573 , 0.71910112, 0.72316384])

In [18]:
cross_val_score(logmodel, X,y, cv=5, scoring='accuracy').mean()

0.6783406335301212

In [19]:
df.head()

,Survived,Pclass,Sex,Embarked
0,0,3,male,S
1,1,1,female,C
2,1,3,female,S
3,1,1,female,S
4,0,3,male,S


In [20]:
from sklearn.preprocessing import OneHotEncoder
ohe  =  OneHotEncoder(sparse=False)

In [21]:
ohe.fit_transform(df[['Sex']])

array([[0., 1.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [0., 1.],
       [0., 1.]])

In [22]:
ohe.categories_

[array(['female', 'male'], dtype=object)]

In [23]:
ohe.fit_transform(df[['Embarked']])

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [24]:
ohe.categories_

[array(['C', 'Q', 'S'], dtype=object)]

In [25]:
# pipeline 

X =  df.drop('Survived', axis ='columns')


In [26]:
X.head()

,Pclass,Sex,Embarked
0,3,male,S
1,1,female,C
2,3,female,S
3,1,female,S
4,3,male,S


In [27]:
from sklearn.compose import make_column_transformer

In [28]:
transform  = make_column_transformer((OneHotEncoder(),['Sex','Embarked']), remainder='passthrough')

In [29]:
transform.fit_transform(X)

array([[0., 1., 0., 0., 1., 3.],
       [1., 0., 1., 0., 0., 1.],
       [1., 0., 0., 0., 1., 3.],
       ...,
       [1., 0., 0., 0., 1., 3.],
       [0., 1., 1., 0., 0., 1.],
       [0., 1., 0., 1., 0., 3.]])

In [30]:
from sklearn.pipeline import make_pipeline

In [31]:
pipe  =  make_pipeline(transform, logmodel)

In [32]:
cross_val_score(pipe, X, y, cv=5, scoring='accuracy')

array([0.76404494, 0.79213483, 0.76966292, 0.75280899, 0.78531073])

In [33]:
cross_val_score(pipe, X, y, cv=5, scoring='accuracy').mean()

0.7727924839713071

In [56]:
X_dat  = X.sample(5, random_state=98)
X_dat

,Pclass,Sex,Embarked
374,3,female,S
530,2,female,S
288,2,male,S
850,3,male,S
217,2,male,S


In [50]:
pipe.fit(X,y)
#logmodel.fit(X,y)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['Sex', 'Embarked'])])),
                ('logisticregression', LogisticRegression())])

In [40]:
pipe.predict(X_dat)

array([1, 1, 1, 0, 0], dtype=int64)